In [20]:
import os
import pandas as pd
import json
import pandas as pd

# Load the JSON file
def cocoJsonToDataFrame(folder_name,get_foleder_name=None):
    if get_foleder_name is None:
        get_foleder_name = folder_name
    
    with open(get_foleder_name+'_annotations.coco.json') as f:
        coco_data = json.load(f)

    # Extract relevant information
    image_id_to_path = {image['id']: image['file_name'] for image in coco_data['images']}
    image_id_to_size = {image['id']: (image['width'], image['height']) for image in coco_data['images']}
    annotations = coco_data['annotations']

    # Create lists to store the extracted information
    image_paths = []
    class_ids = []
    bboxes = []
    image_widths = []
    image_heights = []

    for annotation in annotations:
        image_id = annotation['image_id']
        image_paths.append(image_id_to_path[image_id])
        class_ids.append(annotation['category_id'])
        bbox = annotation['bbox']
        # Convert COCO bbox format (x, y, width, height) to (x_min, y_min, x_max, y_max)
        x_min, y_min, width, height = bbox
        x_max = x_min + width
        y_max = y_min + height
        bboxes.append((x_min, y_min, x_max, y_max))
        image_width, image_height = image_id_to_size[image_id]
        image_widths.append(image_width)
        image_heights.append(image_height)

    # Create the DataFrame
    data = pd.DataFrame({
        'image_path': image_paths,
        'class_id': class_ids,
        'bbox': bboxes,
        'image_width': image_widths,
        'image_height': image_heights
    })

  

    # Directory where YOLO label files will be saved
    label_dir = "labels/"+folder_name
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    


    for idx, row in data.iterrows():
        image_path = row['image_path']
        class_id = 1#row['class_id']
        bbox = row['bbox']
        image_width = row['image_width']
        image_height = row['image_height']
        
        # Convert the bounding box to YOLO format
        x_center, y_center, width, height = convert_bbox_to_yolo(image_width, image_height, bbox)
        
        # Create a label file with the same name as the image (but with .txt extension)
        label_file = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
        label_path = os.path.join(label_dir, label_file)
        
        # Write the YOLO format label to the file
        with open(label_path, 'w') as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


    for idx, row in data.iterrows():
            
        image_path = row['image_path']
        class_id = 1#row['class_id']
        bbox = row['bbox']
        image_width = row['image_width']
        image_height = row['image_height']
        
        # Convert the bounding box to YOLO format
        x_center, y_center, width, height = convert_bbox_to_yolo(image_width, image_height, bbox)
        
        # Create a label file with the same name as the image (but with .txt extension)
        label_file = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
        label_path = os.path.join(label_dir, label_file)
        
        # Write the YOLO format label to the file
        with open(label_path, 'w') as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    import shutil

    # Assuming your images are in a different folder, copy them to the correct folder
    image_dir = "images/"+folder_name

    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    for idx, row in data.iterrows():
        image_path = row['image_path']
        image_path = os.path.join(get_foleder_name, image_path)
        shutil.copy(image_path, image_dir)
    print("Done")

        
cocoJsonToDataFrame("train/")
cocoJsonToDataFrame("val/",get_foleder_name="valid/")
cocoJsonToDataFrame("test/",get_foleder_name="test/")

def convert_bbox_to_yolo(image_width, image_height, bbox):
    xmin, ymin, xmax, ymax = bbox
    x_center = (xmin + xmax) / 2.0 / image_width
    y_center = (ymin + ymax) / 2.0 / image_height
    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height
    return x_center, y_center, width, height

Done
Done
Done


In [ ]:
for idx, row in data.iterrows():
    image_path = row['image_path']
    class_id = row['class_id']
    bbox = row['bbox']
    image_width = row['image_width']
    image_height = row['image_height']
    
    # Convert the bounding box to YOLO format
    x_center, y_center, width, height = convert_bbox_to_yolo(image_width, image_height, bbox)
    
    # Create a label file with the same name as the image (but with .txt extension)
    label_file = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    label_path = os.path.join(label_dir, label_file)
    
    # Write the YOLO format label to the file
    with open(label_path, 'w') as f:
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
import shutil

# Assuming your images are in a different folder, copy them to the correct folder
image_dir = "images/train/"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

for idx, row in data.iterrows():
    image_path = row['image_path']
    shutil.copy(image_path, image_dir)
